In [2]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()

huggingface_model = HuggingFaceModel(
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39",
    env={
        "HF_MODEL_ID": "bhadresh-savani/distilbert-base-uncased-emotion",
        "HF_TASK": "text-classification"
    },
    role = role
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    endpoint_name='prj-text-endpoint'
)


------!

In [2]:
data = {
    "inputs": "What the hell is going on!"
}
predictor.predict(data)



[{'label': 'anger', 'score': 0.9456784725189209}]

In [3]:
import json
f = open('data1.json')
data = json.load(f)
texts = data['texts']

result = []
for t in texts:
    if "content" in t.keys():
        model_data = {
            "inputs": t["content"]
        }
        prediction = predictor.predict(model_data)
        max_tup = (prediction[0]['label'], prediction[0]['score'])
        result.append({"text": t['content'], "sentiment": max_tup})
print(result)
        


[{'text': 'ive enjoyed being able to slouch about relax and unwind and frankly needed it after those last few weeks around the end of uni and the expo i have lately started to find myself feeling a bit listless which is never really a good thing', 'sentiment': ('sadness', 0.9987390637397766)}, {'text': 'i try to be nice though so if you get a bitchy person on the phone or at the window feel free to have a little fit and throw your pen at her face', 'sentiment': ('joy', 0.6493014693260193)}, {'text': 'i guess that feeling is what im really getting all nostalgic about', 'sentiment': ('love', 0.9950023293495178)}, {'text': 'i have to put on a mask when i come into work i have to suppress all of the emptiness i feel inside the pain and loneliness the bitter and jaded woman that i really am', 'sentiment': ('anger', 0.9948829412460327)}, {'text': 'i dunno i just feel scared to walk in after awhile it got ok but when i walk into this room w poorly erected brick walls separating the room into 

In [6]:
predictor.delete_endpoint()

In [1]:
#below is image analysis

from sagemaker.huggingface import HuggingFaceModel
import sagemaker
import boto3
role = sagemaker.get_execution_role()

huggingface_model = HuggingFaceModel(
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39',
    env={
        "HF_MODEL_ID": "openai/clip-vit-base-patch32",
        "HF_TASK": "zero-shot-image-classification"
    },
    role=role,
)

predictor_image = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name='prj-img-endpoint3'
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
------!

In [2]:
labels2 = ['joy','anger', 'love', 'sadness', 'fear', 'surprise']
import json

from PIL import Image

# Example request
image_url = "https://upload.cc/i1/2024/11/27/beCX9Z.jpeg"


url = "https://images.unsplash.com/photo-1634294510755-e7e3b6c25136?q=80&w=2070&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"

candidate_labels2 = ['joy','anger', 'love', 'sadness', 'fear', 'surprise']


data = {
    "inputs": url,
    "parameters": {"candidate_labels": candidate_labels2}
}


# Make a prediction
result = predictor_image.predict(data)
print(result)



[{'score': 0.29346901178359985, 'label': 'joy'}, {'score': 0.22167757153511047, 'label': 'love'}, {'score': 0.1455320119857788, 'label': 'surprise'}, {'score': 0.14531372487545013, 'label': 'anger'}, {'score': 0.12865515053272247, 'label': 'sadness'}, {'score': 0.06535252183675766, 'label': 'fear'}]


In [ ]:
# Don't forget to delete the endpoint when you're done
predictor_image.delete_endpoint()

In [38]:
#image caption / labels

from sagemaker.huggingface import HuggingFaceModel
import sagemaker
import boto3


huggingface_model_labels = HuggingFaceModel(
    transformers_version='4.37.0',
    pytorch_version='2.1.0',
    py_version='py310',
    env={
        "HF_MODEL_ID": "Salesforce/blip-image-captioning-large",
        "HF_TASK": "image-to-text"
    },
    role=role,
)

predictor_label = huggingface_model_labels.deploy(
    initial_instance_count=1, # number of instances
    instance_type='ml.m5.large', # ec2 instance type
    endpoint_name='prj-img-label3'
)

-------!

In [43]:
url = "https://images.unsplash.com/photo-1634294510755-e7e3b6c25136?q=80&w=2070&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"
image_url = "https://upload.cc/i1/2024/11/27/beCX9Z.jpeg"
data = {
    "inputs": url,
    "parameters": {"prompt": "A picture of"}
}


# Make a prediction
result = predictor_label.predict(data)
print(result)


[{'generated_text': 'a picture of a woman in a dress standing on a rocky beach'}]


In [37]:
predictor_label.delete_endpoint()